In [1]:
import gym
import pylab
import numpy as np

# Inverse Reinforcement Learning

The Mountain Car problem is "on a one-dimensional track, positioned between two “mountains”. The goal is to drive up the mountain on the right; however, the car’s engine is not strong enough to scale the mountain in a single pass. Therefore, the only way to succeed is to drive back and forth to build up momentum."

<img src='https://miro.medium.com/max/1104/1*JjBfoFrKCoBxlraVZaEshw.jpeg'>

The car’s state, at any point in time, is given by a vector containing its horizonal position and velocity. The car commences each episode stationary, at the bottom of the valley between the hills (at position, x, approximately -0.5), and the episode ends when either the car reaches the flag (position > 0.5) or after 200 moves.

Your state, s, is a 2-dim vector that will be (position, velocity) or (x,v) of (-0.5,0) in the beginning because the velocity is 0 and x = -0.5 is the valley between the two mountains which is left of center (there is about a distance of 1.1 in front of you and a distance of -0.7 behind you). Only the x-axis is part of the state, the elevation (y-axis) and gravity is implicit. 

The velocity is positive is you are moving forward to the right, and negative if you are moving to the left. 

action 0 is do nothing, action 1 is run the car in reverse towards the left mountain located at `state[0]=-1.2`,
action 2 is run the car forward towards `state[0]=0.6`


In [2]:
# # MACROS
Push_Left = 0
No_Push = 1
Push_Right = 2

In [3]:
# Initialize environment and reset it to start at valley with 0 velocity
env = gym.make('MountainCar-v0')
env.reset() # array([-0.44943939,  0.        ])

# experiment with constantly moving in one direction or the other to see what happens
const_action = No_Push
for t in range(100):
    update = env.step(const_action)
    if t % 5 == 0:
        print(update)
    
# if you do nothing you jus swing side to side, but your position stays in (-0.64,-0.4)
# your velocity stays in (-0.01,+0.01)

(array([-4.90682084e-01, -2.48327461e-04]), -1.0, False, {})
(array([-0.49552003, -0.00142585]), -1.0, False, {})
(array([-0.5055381, -0.0023409]), -1.0, False, {})
(array([-0.51889013, -0.00282428]), -1.0, False, {})
(array([-0.53311171, -0.00278615]), -1.0, False, {})
(array([-0.54557669, -0.00223363]), -1.0, False, {})
(array([-0.55398543, -0.00126931]), -1.0, False, {})
(array([-5.56789245e-01, -7.13889865e-05]), -1.0, False, {})
(array([-0.55347239,  0.00113966]), -1.0, False, {})
(array([-0.54464503,  0.002141  ]), -1.0, False, {})
(array([-0.53193317,  0.00274771]), -1.0, False, {})
(array([-0.51768229,  0.00284717]), -1.0, False, {})
(array([-0.50452399,  0.00242087]), -1.0, False, {})
(array([-0.49488668,  0.00154805]), -1.0, False, {})
(array([-4.90546092e-01,  3.90223111e-04]), -1.0, False, {})
(array([-0.49230084, -0.00083937]), -1.0, False, {})
(array([-0.49982818, -0.00191456]), -1.0, False, {})
(array([-0.51174214, -0.00263702]), -1.0, False, {})
(array([-0.52584533, -0.

In [4]:
# Initialize environment and reset it to start at valley with 0 velocity
env = gym.make('MountainCar-v0')
env.reset() # array([-0.44943939,  0.        ])

# experiment with constantly moving in one direction or the other to see what happens
# constantly do nothing
const_action = No_Push
for t in range(100):
    update = env.step(const_action)
    if t % 5 == 0:
        print(update)
    
# if you do nothing you jus swing side to side, but your position stays in (-0.64,-0.4)
# your velocity stays in (-0.01,+0.01)

(array([-5.80546953e-01,  4.28201021e-04]), -1.0, False, {})
(array([-0.57220265,  0.00245964]), -1.0, False, {})
(array([-0.55491257,  0.00404139]), -1.0, False, {})
(array([-0.53184862,  0.00488026]), -1.0, False, {})
(array([-0.50726168,  0.00481851]), -1.0, False, {})
(array([-0.48568991,  0.00386784]), -1.0, False, {})
(array([-0.47110389,  0.0022067 ]), -1.0, False, {})
(array([-4.66174844e-01,  1.42353636e-04]), -1.0, False, {})
(array([-0.47180201, -0.00194793]), -1.0, False, {})
(array([-0.48695862, -0.0036825 ]), -1.0, False, {})
(array([-0.50886803, -0.00474114]), -1.0, False, {})
(array([-0.53349632, -0.00492546]), -1.0, False, {})
(array([-0.55629752, -0.00420062]), -1.0, False, {})
(array([-0.57307017, -0.0027031 ]), -1.0, False, {})
(array([-0.58073838, -0.00071104]), -1.0, False, {})
(array([-0.5779022 ,  0.00141063]), -1.0, False, {})
(array([-0.56507889,  0.00327496]), -1.0, False, {})
(array([-0.54461455,  0.00453905]), -1.0, False, {})
(array([-0.52027243,  0.004966

In [5]:
# Initialize environment and reset it to start at valley with 0 velocity
env = gym.make('MountainCar-v0')
env.reset() # array([-0.44943939,  0.        ])

# experiment with constantly moving in one direction or the other to see what happens
# constantly go right
const_action = Push_Right
for t in range(100):
    update = env.step(const_action)
    if t % 5 == 0:
        print(update)
    
# if you constantly go right,  your position stays in (-0.52,-0.24)
# your velocity stays in (-0.011,+0.011)

(array([-4.35774143e-01,  3.51454974e-04]), -1.0, False, {})
(array([-0.42892149,  0.00202079]), -1.0, False, {})
(array([-0.41468882,  0.0033319 ]), -1.0, False, {})
(array([-0.395581  ,  0.00405957]), -1.0, False, {})
(array([-0.37490578,  0.00409019]), -1.0, False, {})
(array([-0.3561608 ,  0.00343614]), -1.0, False, {})
(array([-0.34243423,  0.00222232]), -1.0, False, {})
(array([-0.33593213,  0.00065469]), -1.0, False, {})
(array([-0.33768203, -0.00101588]), -1.0, False, {})
(array([-0.34740854, -0.00252646]), -1.0, False, {})
(array([-0.36356573, -0.00363307]), -1.0, False, {})
(array([-0.38353143, -0.00414438]), -1.0, False, {})
(array([-0.40397919, -0.00395784]), -1.0, False, {})
(array([-0.42141405, -0.00308839]), -1.0, False, {})
(array([-0.43279422, -0.00167634]), -1.0, False, {})
(array([-4.36107325e-01,  3.33797158e-05]), -1.0, False, {})
(array([-0.43076386,  0.00173715]), -1.0, False, {})
(array([-0.41771379,  0.00313254]), -1.0, False, {})
(array([-0.39925896,  0.003977

In [6]:
# Initialize environment and reset it to start at valley with 0 velocity
env = gym.make('MountainCar-v0')
env.reset() # array([-0.44943939,  0.        ])

# experiment with constantly moving in one direction or the other to see what happens
# constantly go left
const_action = Push_Left
for t in range(100):
    update = env.step(const_action)
    if t % 5 == 0:
        print(update)
    
# if you constantly go left,  your position stays in (-0.85,-0.48)
# your velocity stays in (-0.015,+0.011)

(array([-0.40664363, -0.00187245]), -1.0, False, {})
(array([-0.44317188, -0.01077461]), -1.0, False, {})
(array([-0.51902217, -0.01774168]), -1.0, False, {})
(array([-0.62034497, -0.02145069]), -1.0, False, {})
(array([-0.72882767, -0.02136434]), -1.0, False, {})
(array([-0.82663192, -0.01798935]), -1.0, False, {})
(array([-0.90061614, -0.01246169]), -1.0, False, {})
(array([-0.9433909 , -0.00586423]), -1.0, False, {})
(array([-0.95186324,  0.00110282]), -1.0, False, {})
(array([-0.92553048,  0.00800718]), -1.0, False, {})
(array([-0.86607549,  0.01436238]), -1.0, False, {})
(array([-0.77855488,  0.0193428 ]), -1.0, False, {})
(array([-0.67324759,  0.02178924]), -1.0, False, {})
(array([-0.56606887,  0.02068764]), -1.0, False, {})
(array([-0.47575798,  0.015847  ]), -1.0, False, {})
(array([-0.41885329,  0.00810581]), -1.0, False, {})
(array([-0.40561892, -0.00107687]), -1.0, False, {})
(array([-0.43838899, -0.01007077]), -1.0, False, {})
(array([-0.51134165, -0.01726094]), -1.0, Fals

in the update, the tuple  (array([-0.66511328, -0.01531442]), -1.0, False, {})
 contains the (state, reward, end_of_episode_bool, meta_data)
the meta_data is empty for this environment, there is a negative reward for every timestep that 
 you have not reached the goal yet. 

In [8]:
# load the expert 20 demonstrations
#expert_demo = np.load('/shared/Carson/RL/lets-do-irl/mountaincar/maxent/expert_demo/expert_demo.npy')
expert_demo = np.load('Data/expert_demo.npy')
print(expert_demo.shape)
# (number of demonstrations, length of demonstrations, states and actions of demonstrations)
print(expert_demo[0,60,:], expert_demo[0,0,:].shape)
# as you can see from step 60 of the first example, the best strategy is to first accelerate backwards into
# the < -0.8 range into order to gain speed going right into the valley. 

(20, 130, 3)
[-0.90691623 -0.02983074  0.        ] (3,)


In [11]:
'''
To descretize the state space, we separate the range of possible continuous positions x and
continuous velocities y into 20 bins (one_feature)
'''

one_feature = 20 # number of state per one feature

env_low = env.observation_space.low     
env_high = env.observation_space.high   
env_distance = (env_high - env_low) / one_feature  

print(env_low, env_high, env_distance, env_distance[0]*20, env_distance[1]*20)
# the range of x is 1.8 and velocity is 0.14 

[-1.2  -0.07] [0.6  0.07] [0.09  0.007] 1.8000000715255737 0.14000000432133675


In [13]:
'''
using this bin size we descretize the expert demonstrations
The difference between expert_demo and demonstrations is that
demonstrations consists of discreet integers instead of continuous values
for it's states



here I used n for sample index and t for timestep index
'''
def idx_state(state):
    '''
    this function converts a continuous state vector of
    2-dim into a discrete index
    
    by assigning each state and index of state_idx = position_idx + velocity_idx * one_feature
    we make sure that (position_idx, velocity_idx) = (2,3) and (3,2) map to different integers
    The first 20 elements of state_idx go to position_idx = 0 - 19, velocity_idx = 0, etc
    '''
    position_idx = int((state[0] - env_low[0]) / env_distance[0])
    velocity_idx = int((state[1] - env_low[1]) / env_distance[1])
    state_idx = position_idx + velocity_idx * one_feature
    return state_idx

demonstrations = np.zeros((len(expert_demo), len(expert_demo[0]), 3))

for n in range(len(expert_demo)):
    
    for t in range(len(expert_demo[0])):

        state_idx = idx_state(expert_demo[n][t])

        demonstrations[n][t][0] = state_idx
        demonstrations[n][t][1] = expert_demo[n][t][2] 

Q-Learning is a “model-free, off-policy” RL algorithm.

“model-free” because we are not trying to build a pre-determined model of our environment (which may or may not be accurate)

“off-policy” because our behavior in interacting with (or exploring) the environment may be unrelated to what we believe to be the best optimal at the time of taking the action.

You might think of Q-learning as updating a table where each row is a different state and each column is an action that can be taken in that state and where they intersect is the Q-value for that state action pair. the Q-table maps various (state, action) pairs with the Q-value (the expected sum of discounter future rewards when taking action a in state s and behaving optimally thereafter)

It doesnt have to be a square table like this though, Q just has to map some state or state-action to some Q-value

In the above example you can see that the pattern is that we observe some state, s, take an action a, and receive a new state s’, then repeat. 

The update rule is the weighted average of the old Q(s,a) value and the Q value implied by the new observation. That is, the sum of (i) the immediate reward and (ii) the expected discounted reward received from the new state onwards, assuming you always choose the optimal action.

Q’(s,a) = (1 — alpha) * Q(s, a) + alpha *(r + gamma * Q(s’, argmax a’ : Q(s’, a’)))

where argmax a’ : Q(s’, a’) is the action  a’ that has the highest current Q value among our row of states s’

This equation for update is equivalently written 

𝑄(𝑠,𝑎)←𝑄(𝑠,𝑎)+𝛼(𝑟+𝛾*max_𝑎′𝑄(𝑠′,𝑎′)−𝑄(𝑠,𝑎)) 


In [14]:
def update_q_table(state, action, reward, next_state):
    ''' 
    The Q-learning update rule
    𝑄(𝑠,𝑎)←𝑄(𝑠,𝑎)+𝛼(𝑟+𝛾*max_𝑎′𝑄(𝑠′,𝑎′)−𝑄(𝑠,𝑎))
    '''
    q_1 = q_table[state][action]
    q_2 = reward + gamma * max(q_table[next_state])
    q_table[state][action] += q_learning_rate * (q_2 - q_1)

In [ ]:
n_states = 

feature_matrix = np.eye((n_states)) # (400, 400)